In [ ]:
import qiskit
qiskit.__qiskit_version__

In [ ]:
from qiskit import *
from qiskit import Aer
from qiskit.circuit.library import QFT
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import HHL, NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def create_eigs(matrix, num_ancillae, num_time_slices, negative_evals):
    ne_qfts = [None, None]
    if negative_evals:
        num_ancillae += 1
        ne_qfts = [QFT(num_ancillae - 1), QFT(num_ancillae - 1).inverse()]

    return EigsQPE(MatrixOperator(matrix=matrix),
                   QFT(num_ancillae).inverse(),
                   num_time_slices=num_time_slices,
                   num_ancillae=num_ancillae,
                   expansion_mode='suzuki',
                   expansion_order=2,
                   evo_time=None,  # This is t, can set to: np.pi*3/4
                   negative_evals=negative_evals,
                   ne_qfts=ne_qfts)

In [ ]:
#u at t=0
u_0= np.zeros(41) 

#initial condition
for i in range(0,41): 
    if i<=10:
        u_0[i] = 1
    else:
        u_0[i] = 2

A = np.zeros([41,41],int)
A = np.matrix(A)
#A matrix
for i in range(0,41):
    A[i,i:i+3] += 1

In [ ]:
#u at t=1
u_1 = NumPyLSsolver(A, u_0).run()
print("Classical Solution:\t", np.round(u_1['solution'], 5))

In [ ]:
# u at t=2
u_2 = NumPyLSsolver(A, u_1['solution']).run()
print("Classical Solution:\t", np.round(u_2['solution'], 5))

In [ ]:
A1 = A[0:4,0:4]
u_01 = u_0[0:4]
orig_size = len(u_01)
A1, u_01, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(A1, u_01)

# Initialize eigenvalue finding module
eigs = create_eigs(A1, 3, 50, False)
num_q, num_a = eigs.get_register_sizes()

# Initialize initial state module
init_state = Custom(num_q, state_vector=u_01)

# Initialize reciprocal rotation module
reci = LookupRotation(negative_evals=eigs._negative_evals, evo_time=eigs._evo_time)

algor = HHL(A1, u_01, truncate_powerdim, truncate_hermitian, eigs,
           init_state, reci, num_q, num_a, orig_size)


In [ ]:
print('Number of qubits=' + str(num_q)+',''Number of ancillia=' + str(num_a))

In [ ]:
#Quantum Circuit of HHL Algorithm implementation
qc=HHL.construct_circuit(algor,measurement = False)
qc.draw('mpl')

In [ ]:
u1_num=np.array(u_1['solution'])

In [ ]:
#x values
x= np.linspace(0,2,41

In [ ]:
#u at t=1 using HHL Algorithm
u_1_q = algor.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
print("Solution:\t\t", (np.round(u_1_q['solution'],5)))

In [ ]:
#Analytical solution is u=u0(x-Ct) and C=1 

#for t=1 u=u0(x-1)
u1_ana= np.zeros(41)
u2_ana= np.zeros(41) 
u3_ana= np.zeros(41) 
u4_ana= np.zeros(41) 
u5_ana= np.zeros(41) 
u6_ana= np.zeros(41) 
u7_ana= np.zeros(41) 
u8_ana= np.zeros(41) 
u9_ana= np.zeros(41) 

u1_ana = u_0*[i-1 for i in x] #t=1
u2_ana = u_0*[i-2 for i in x] #t=2
u3_ana = u_0*[i-3 for i in x] #t=3
u4_ana = u_0*[i-4 for i in x] #t=4
u5_ana = u_0*[i-5 for i in x] #t=5
u6_ana = u_0*[i-6 for i in x] #t=6
u7_ana = u_0*[i-7 for i in x] #t=7
u8_ana = u_0*[i-8 for i in x] #t=8
u9_ana = u_0*[i-9 for i in x] #t=9


#L1 norm of analytical solution
norm_u1_ana = sum(abs(u1_ana))
norm_u2_ana = sum(abs(u2_ana))
norm_u3_ana = sum(abs(u3_ana))
norm_u4_ana = sum(abs(u4_ana))
norm_u5_ana = sum(abs(u5_ana))
norm_u6_ana = sum(abs(u6_ana))
norm_u7_ana = sum(abs(u7_ana))
norm_u8_ana = sum(abs(u8_ana))
norm_u9_ana = sum(abs(u9_ana))


#L1 norm of numerical solution
norm_u1_num = sum(abs(u1_num))


In [ ]:
#plot l1 norm of analytical solution
plt.plot([norm_u1_ana,norm_u2_ana,norm_u3_ana,norm_u4_ana,norm_u5_ana,norm_u6_ana,norm_u7_ana,norm_u8_ana,norm_u9_ana])

In [ ]:
#Noise Simulation
import numpy as np
from qiskit import execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_histogram

# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import thermal_relaxation_error

In [ ]:
qc.measure_all()
# Ideal simulator and execution
ideal_simulator = QasmSimulator()
job = execute(qc, ideal_simulator)
result_ideal = job.result()
plot_histogram(result_ideal.get_counts(0))

In [ ]:
# Example error probabilities
p_reset = 0.03
p_meas = 0.1
p_gate1 = 0.05

# QuantumError objects
error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
error_gate2 = error_gate1.tensor(error_gate1)

# Add errors to noise model
noise_bit_flip = NoiseModel()
noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])

print(noise_bit_flip)

In [ ]:
# Run the noisy simulation
noisy_simulator = QasmSimulator(noise_model=noise_bit_flip)
job = execute(qc, noisy_simulator)
result_bit_flip = job.result()
counts_bit_flip = result_bit_flip.get_counts(0)

# Plot noisy output
plot_histogram(counts_bit_flip)

In [ ]:
# T1 and T2 values for qubits 0-3
T1s = np.random.normal(50e3, 10e3, 4) # Sampled from normal distribution mean 50 microsec
T2s = np.random.normal(70e3, 10e3, 4)  # Sampled from normal distribution mean 50 microsec

# Truncate random T2s <= T1s
T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(4)])

# Instruction times (in nanoseconds)
time_u1 = 0   # virtual gate
time_u2 = 50  # (single X90 pulse)
time_u3 = 100 # (two X90 pulses)
time_cx = 300
time_reset = 1000  # 1 microsecond
time_measure = 1000 # 1 microsecond

# QuantumError objects
errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
                for t1, t2 in zip(T1s, T2s)]
errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
                  for t1, t2 in zip(T1s, T2s)]
errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
              for t1, t2 in zip(T1s, T2s)]
errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
              for t1, t2 in zip(T1s, T2s)]
errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
              for t1, t2 in zip(T1s, T2s)]
errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
             thermal_relaxation_error(t1b, t2b, time_cx))
              for t1a, t2a in zip(T1s, T2s)]
               for t1b, t2b in zip(T1s, T2s)]

# Add errors to noise model
noise_thermal = NoiseModel()
for j in range(4):
    noise_thermal.add_quantum_error(errors_reset[j], "reset", [j])
    noise_thermal.add_quantum_error(errors_measure[j], "measure", [j])
    noise_thermal.add_quantum_error(errors_u1[j], "u1", [j])
    noise_thermal.add_quantum_error(errors_u2[j], "u2", [j])
    noise_thermal.add_quantum_error(errors_u3[j], "u3", [j])
    for k in range(4):
        noise_thermal.add_quantum_error(errors_cx[j][k], "cx", [j, k])

print(noise_thermal)

In [ ]:
# Run the noisy simulation
thermal_simulator = QasmSimulator(noise_model=noise_thermal)
job = execute(qc, thermal_simulator)
result_thermal = job.result()
counts_thermal = result_thermal.get_counts(0)

# Plot noisy output
plot_histogram(counts_thermal)